## 웹 크롤링

### 인터넷 접속 라이브러리 추가

In [13]:
from urllib.request import urlopen, Request

#도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page =  urlopen(url=url)

    text= page.read().decode('utf-8')
    text=text[text.find(f'>{city}</a'):]
    # 기온 가져오기
    for i in range(7):
        text= text[text.find('<td>')+1:]
        
    start=3
    end=text.find('</td>')
    current_temp= text[start:end]
    print(f'{city}의 현재기온은 {current_temp}˚C 입니다')

    #습도 가져오기
    for i in range(3):
        text= text[text.find('<td>')+1:]
        
    start=3
    end=text.find('</td>')
    current_temp= text[start:end]
    print(f'{city}의 현재 습도는 {current_temp}% 입니다.')


get_weather('부산')

부산의 현재기온은 9.7˚C 입니다
부산의 현재 습도는 36% 입니다.


### OpenAIP 크롤링

- 공공데이터 포털
    - http://www.data.go.kr
    - 부산광역시 버스정보안내기 현황

In [35]:
import requests
from urllib.request import urlopen, Request
from urllib.parse import quote,unquote,urlencode # 한글 URLencode 변환하는 함수
import json
import ssl

def getRequestUrl(url):
    req = Request(url)

    try:
        res=urlopen(req)
        if res.getcode()==200:
            return res.read().decode('utf-8')
    except Exception as e:
        print()
        return None

# stationName 정류소이름 
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(

        {
            'serviceKey' : 'jmhj4AySTfuqupZqy/HnxlkTQs+iGPx391UTg2b1LoaAHtc6FzkpEnmerQYc5Oxvkb2klHsgnoTKB3nI0XllFg==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'resultType' : type,
            'stationLoc' : stationName

        }
    )
    total_url= api_url+ queryString
    response = requests.get(total_url, verify= False)
    return response.text
 

result = getDataPortalSearch('롯데백화점', 'json')
json_data =  json.loads(result)
station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

for item in station_data:
    print(item)


{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06030', 'stationLoc': '롯데백화점동래점.명륜역', 'lat': '35.21276314', 'lng': '129.0781263', 'addr': '동래구 온천1동 474-25', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06046', 'stationLoc': '롯데백화점동래점', 'lat': '35.2101971', 'lng': '129.0778307', 'addr': '동래구 온천2동 503-46', 'insYear': '', 'dataDay': '2023-02-03'}


c:\source\studyPython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시



In [36]:
import folium

if len(station_data) > 0:
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']
m = folium.Map(location=[center_lat, center_lng], zoom_start=13)

for item in station_data:
    folium.Marker(location=[item['lat'], item['lng']], popup=item['stationLoc']).add_to(m)
m

ModuleNotFoundError: No module named 'folium'